In [1]:
from pathlib import Path
import os
import pandas as pd
Path.cont = lambda x: list(os.scandir(x)) # scandir gives all content of a dir 
filedir = 'RIVM_data'
files = Path.cont(filedir)

def generate_df(filedir,filename,i):
    rename = {'Category':'Gemeente',
            'Aantal per 100.000 inwoners':'Aantal per 100000 inwoners'}
    drop = ['id','Indicator','Gemnr']
    if len(filename.name) > 30:
        df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',delimiter=';')
    else:
        df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',skiprows=1)
    for old,new in rename.items():
        if old in df.columns:
            df.rename(columns={old: new},inplace=True)
    for d in drop:
        if d in df.columns:
            df = df.drop([d], axis=1) 
    df['Datum'] = filename.name[:10]
    df['Count'] = str(i).zfill(2)
    return df

num_csv = len(files)
df = pd.concat([generate_df(filedir,filename,i) for i,filename in enumerate(files)])
df.dropna(axis=0, how='any',subset=['Aantal'], inplace=True)

df = df.sort_values(['Gemeente','Datum'], ascending = (True, False))

df['BevAant'].fillna(method='ffill', axis = 0, inplace=True)
df['Aantal per 100000 inwoners'] = df['Aantal'] / df['BevAant'] * 100000

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [2]:
def clean_df(df):
    df = df.fillna(0)
    df = df.reindex(sorted(df.columns), axis=1)
    df.rename(columns=dict(zip(df.columns,[str(int(c)) if not c[0].isalpha() else c for c in df.columns])),inplace=True)
    return df


df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_aantal = clean_df(df_aantal)


df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_relative = clean_df(df_relative)

In [3]:
# make a growth df
df_growth = df_aantal
temp_columns = df_growth.columns
import numpy as np
def calc_growth(line):
    # line = df_aantal[df_aantal['Gemeente']=='Tilburg'].values[0]
    # growth defined as today/yesterday -1
    # print(line)
    growth = [y/x-1 if x>4 else 0 for x,y in zip(line[:-1],line[1:-1])]
    # moving average over the growth
    smoothed_growth = [(x1+x2+x3+x4)/3 for x1,x2,x3,x4 in zip(growth,growth[1:],growth[2:],growth[3:])]
    # growth defined as today/4 days ago -1
    alternative_growth = [y/x-1 if x>4 else 0 for x,y in zip(line[:-1],line[3:-1])]
    result = [0]*(len(line)-len(alternative_growth)-1) + alternative_growth + [line[-1]]
    # print(len(line), len(growth), len(smoothed_growth),len(alternative_growth), len(result))
    return result

df_growth = pd.DataFrame.from_records(df_growth.apply(calc_growth,axis=1))
df_growth.rename(columns=dict(zip(df_growth.columns,temp_columns)),inplace=True)

In [21]:
import altair as alt
import pandas as pd
from vega_datasets import data
url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

columns = [str(day) for day in range(0, num_csv)]

slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend):
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns),
        default = '0'
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        tooltip=['properties.statnaam:N','18:Q'],
        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='spectral', type='symlog',domain=[max(df[str(num_csv-1)]),0])),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=300,
        height=400
    ).transform_filter(
        select_year
    )    
    return chart


In [22]:
alt.hconcat(
    serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief'),serve_chart(df_growth, 'groei')
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.HConcatChart(...)

20 18 15 16


In [52]:
# inspect individual cities
x = list(df_aantal[df_aantal['Gemeente']=="Amsterdam"].values[0][:-1])
y = list(range(len(x)))
df = pd.DataFrame(x,y).reset_index()
print(df.columns)
df.rename(columns=dict(zip(df.columns,[str(c) for c in df.columns])),inplace=True)

alt.Chart(df).mark_point().encode(
    x='index:Q',
    y='0:Q',
)

Index(['index', 0], dtype='object')


alt.Chart(...)

In [16]:
df_growth.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Gemeente
0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.428571,0.750000,0.888889,1.100000,0.928571,1.117647,1.095238,'s-Gravenhage
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.800000,0.750000,0.818182,0.714286,1.142857,0.750000,0.583333,0.500000,'s-Hertogenbosch
2,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,* Bij 112 personen is de woonplaats niet van b...
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,* Bij 137 personen is de woonplaats niet van b...
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,* postcode ontbreekt bij 37 pt en 1 woont in b...


In [7]:
df_aantal.head()

Count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Gemeente
0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,3.0,7.0,7.0,8.0,9.0,10.0,14.0,17.0,21.0,27.0,36.0,44.0,'s-Gravenhage
1,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,5.0,8.0,11.0,14.0,14.0,20.0,24.0,30.0,35.0,38.0,45.0,'s-Hertogenbosch
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,* Bij 112 personen is de woonplaats niet van b...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,* Bij 137 personen is de woonplaats niet van b...
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,* postcode ontbreekt bij 37 pt en 1 woont in b...


In [8]:
df_growth[df_growth['Gemeente']=='Tilburg']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Gemeente
299,0,0,0,0,0.0,0.0,0.0,1.9,2.5,0.851852,1.310345,0.619048,0.36,0.253731,0.294118,0.397059,0.416667,0.511364,0.621053,Tilburg
